In [30]:
import os, sys

sys.path.append('c:\\Projects\\taxi_predicts\\src\\')

In [31]:
import config as config

In [32]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

#we fetch raw data for the last 28days to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

print(fetch_data_from)

current_date=Timestamp('2024-03-20 09:00:00')
2024-02-21 09:00:00


In [33]:
#we cannot import the current data from data warehouse
# so we simulate the data

from data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    #we simulate production data by sampling historical data from 52weeks ago 

    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)

    print(from_date_)
    print(to_date_)

    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    #shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [34]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

2023-02-22 09:00:00
2023-03-22 09:00:00
File 2023-02 was already in local storage
File 2023-03 was already in local storage


In [35]:
from data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:00<00:00, 312.14it/s]


In [36]:
print(f'{ts_data.head(10)}')

          pickup_hour  rides  pickup_location_id
0 2024-02-21 09:00:00      0                   1
1 2024-02-21 10:00:00      1                   1
2 2024-02-21 11:00:00      0                   1
3 2024-02-21 12:00:00      0                   1
4 2024-02-21 13:00:00      0                   1
5 2024-02-21 14:00:00      0                   1
6 2024-02-21 15:00:00      0                   1
7 2024-02-21 16:00:00      1                   1
8 2024-02-21 17:00:00      0                   1
9 2024-02-21 18:00:00      1                   1


In [37]:
print(f'{ts_data.tail(10)}')

               pickup_hour  rides  pickup_location_id
178070 2024-03-19 23:00:00      1                 265
178071 2024-03-20 00:00:00      6                 265
178072 2024-03-20 01:00:00      4                 265
178073 2024-03-20 02:00:00      0                 265
178074 2024-03-20 03:00:00      2                 265
178075 2024-03-20 04:00:00      3                 265
178076 2024-03-20 05:00:00      5                 265
178077 2024-03-20 06:00:00      9                 265
178078 2024-03-20 07:00:00      3                 265
178079 2024-03-20 08:00:00      7                 265


In [38]:
import hopsworks
#now we connect to the feture_store and feature group to store the data

#connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

#connect to the feature store
feature_store = project.get_feature_store()

#connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description='Time-series data at hourly frequency',
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour'
)



Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549012
Connected. Call `.close()` to terminate connection gracefully.


In [39]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 178080/178080 | Elapsed Time: 00:18 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549012/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2e32636eb30>, None)

In [ ]:
#now we need to automate this so it runs every hour, we do that in github actions